### This is a new version of MLEAP scripts, started in late Aug 2022.
It will combine IProject_MLEAP_ANN and IP_MLEAP script, while improving them.

#### Outline

1. Define a problem, describe business context.
2. Load data, preclean it.
3. EDA: target, features.
4. Descibe train/test split strategy. Show main results and discuss them.
5. Run evth for a few periods. Show feature importance and error analysis.

### 1. Business problem

#### Objective:

#### Metric:

#### Summary of results:


### 2. Load data and preclean it

In [3]:
# 0. Import libraries #

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import os, time, math, re, warnings, random, gc, dill, optuna, pickle, shap
import statsmodels.api as sm
from random import sample

from sklearn.model_selection import train_test_split, KFold, PredefinedSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance
from category_encoders import MEstimateEncoder
from xgboost import XGBRegressor
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

plt.style.use('seaborn-white')
warnings.simplefilter(action='ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
pd.set_option('display.max_columns', 150)
gc.enable()

In [4]:
### target encoding ###
# source: https://www.kaggle.com/code/ryanholbrook/feature-engineering-for-house-prices/notebook

class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=4)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [5]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [6]:
# Functions for Optuna NNs

cv_nn_regularizer = 0.075

def create_snnn4_model_hyperpars(neurons_base=4, l1_reg_rate=0.1, l2_reg_rate=0.1):

    model_snn = Sequential([
        tf.keras.layers.Dense(units=neurons_base*4, activation="selu", kernel_initializer="lecun_normal", 
                              kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate), 
                              input_shape=X_train.shape[1:]),
        tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base/2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        Dense(1)])

    optimizer_adam = tf.keras.optimizers.Adam()
    model_snn.compile(loss= "mean_squared_error" , optimizer=optimizer_adam, metrics=["mean_squared_error"])
    
    return model_snn

def create_snnn4_model(trial):

    # hyperparameter space
    neurons_base = trial.suggest_int("neurons_base", 4, 24, 2)
    l2_reg_rate = trial.suggest_float('l2_regularizer', 0, 0.9)
    l1_reg_rate = trial.suggest_float('l1_regularizer', 0, 0.9)

    model_snn = Sequential([
        tf.keras.layers.Dense(units=neurons_base*4, activation="selu", kernel_initializer="lecun_normal", 
                              kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate), 
                              input_shape=X_train.shape[1:]),
        tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base/2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        Dense(1)])

    optimizer_adam = tf.keras.optimizers.Adam()
    model_snn.compile(loss= "mean_squared_error" , optimizer=optimizer_adam, metrics=["mean_squared_error"])
    
    return model_snn


def create_snnn6_model_hyperpars(neurons_base=4, l1_reg_rate=0.1, l2_reg_rate=0.1):

    model_snn = Sequential([
        tf.keras.layers.Dense(units=neurons_base*4, activation="selu", kernel_initializer="lecun_normal", 
                              kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate), 
                              input_shape=X_train.shape[1:]),
        tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base/2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        Dense(1)])

    optimizer_adam = tf.keras.optimizers.Adam()
    model_snn.compile(loss= "mean_squared_error" , optimizer=optimizer_adam, metrics=["mean_squared_error"])
    
    return model_snn

def create_snnn6_model(trial):

    # hyperparameter space
    neurons_base = trial.suggest_int("neurons_base", 4, 24, 2)
    l2_reg_rate = trial.suggest_float('l2_regularizer', 0, 0.9)
    l1_reg_rate = trial.suggest_float('l1_regularizer', 0, 0.9)

    model_snn = Sequential([
        tf.keras.layers.Dense(units=neurons_base*4, activation="selu", kernel_initializer="lecun_normal", 
                              kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate), 
                              input_shape=X_train.shape[1:]),
        tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        tf.keras.layers.Dense(units=neurons_base/2, activation="selu", kernel_initializer="lecun_normal",
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg_rate, l2=l2_reg_rate)),
        Dense(1)])

    optimizer_adam = tf.keras.optimizers.Adam()
    model_snn.compile(loss= "mean_squared_error" , optimizer=optimizer_adam, metrics=["mean_squared_error"])
    
    return model_snn


def objective_nn4(trial):
    
    tf.keras.backend.clear_session()
    
    with strategy.scope():
        # Generate our trial model.
        model = create_snnn4_model(trial)

        callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=40),
        TFKerasPruningCallback(trial, "val_loss"),
    ]

        # Fit the model on the training data.
        # The TFKerasPruningCallback checks for pruning condition every epoch.
        
        history = model.fit(X_train, y_train, 
                                validation_data=(X_val, y_val),
                                batch_size=2048, 
                                epochs=500, 
                                verbose=0, 
                                callbacks=callbacks)

        # Evaluate the model accuracy on the validation set.
        loss_train = model.evaluate(X_train, y_train, verbose=0)
        loss_val = model.evaluate(X_val_extra, y_val_extra, verbose=0)
        overfit = max(loss_val[1]-loss_train[1], 0)
        return loss_val[1] + cv_nn_regularizer*overfit
    
    
def objective_nn6(trial):
    
    tf.keras.backend.clear_session()
    
    with strategy.scope():
        # Generate our trial model.
        model = create_snnn6_model(trial)

        callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=40),
        TFKerasPruningCallback(trial, "val_loss"),
    ]

        # Fit the model on the training data.
        # The TFKerasPruningCallback checks for pruning condition every epoch.
        
        history = model.fit(X_train, y_train, 
                                validation_data=(X_val, y_val),
                                batch_size=2048, 
                                epochs=500, 
                                verbose=0, 
                                callbacks=callbacks)

        # Evaluate the model accuracy on the validation set.
        loss_train = model.evaluate(X_train, y_train, verbose=0)
        loss_val = model.evaluate(X_val_extra, y_val_extra, verbose=0)
        overfit = max(loss_val[1]-loss_train[1], 0)
        return loss_val[1] + cv_nn_regularizer*overfit


In [7]:
time0 = time.time()

min_prd = 500
windows_width = 5*12
cv_xgb_regularizer=0.2
optuna_xgb_trials = 80
optuna_nn_trials = 100


with open('../input/mleap-46-preprocessed/MLEAP_46_v7.pkl', 'rb') as pickled_one:
    df = pickle.load(pickled_one)
df = df[df.prd.isin(range(min_prd-1, min_prd+windows_width+10))]
df_cnt = df.count()
empty_cols = list(df_cnt[df_cnt<int(df.shape[0]/2)].index)
df.drop(columns=empty_cols, inplace=True)
#display(df.shape, df.head(), df.year.describe(), df.count())

features_miss_dummies = ['amhd', 'BAspr']
for col in features_miss_dummies:
    if col in df.columns:
        df[col+'_miss'] = df[col].isnull().astype(int)


### 3. EDA

### 4. Train/test split strategy

#### Main results

In [8]:
results_df = pd.read_csv('../input/mleap-v49-results/temp_models_reg005_1.csv')
results_df

,Unnamed: 0,min_prd,xgbf_train,xgbf_val,xgbf_test,xgbgs_train,xgbgs_val,xgbgs_test,xgbo_train,xgbo_val,xgbo_test,nn4_train,nn4_val,nn4_test,nn6_train,nn6_val,nn6_test,nn4opt_train,nn4opt_val,nn4opt_test,nn6opt_train,nn6opt_val,nn6opt_test
0,0,100,0.110633,-0.010679,0.005897,0.032282,0.009775,0.016553,0.142414,0.097214,0.017316,0.092751,-0.004731,0.004806,0.077033,0.003310,0.007623,2.368052e-02,-0.006378,4.413288e-03,2.373109e-02,-1.185371e-03,0.006157
1,1,125,0.066434,0.004399,0.015985,0.040227,0.053275,0.020740,0.081517,0.077877,0.018348,0.020937,0.023310,0.014654,0.039157,0.018810,0.009496,-1.010410e-06,-0.000038,-1.945688e-05,-7.328776e-07,-3.910448e-05,-0.000020
2,2,150,0.064235,0.028400,-0.007106,0.108114,0.136225,-0.040268,0.102913,0.141299,-0.040073,0.039123,0.041242,-0.044962,0.046364,0.044968,-0.055987,4.366676e-02,0.040878,-4.501329e-02,5.041909e-02,4.294926e-02,-0.043427
3,3,175,0.073734,0.019790,-0.011655,0.093350,0.065665,0.000453,0.017994,0.013669,-0.002192,0.057370,0.021523,-0.006702,0.064954,0.017231,-0.010337,-4.252701e-06,-0.000010,-4.954723e-07,-7.191610e-09,-6.917556e-07,-0.000008
4,4,200,0.056176,0.048308,0.034120,0.111365,0.141292,0.040536,0.057042,0.089245,0.040346,0.041166,0.071331,0.022324,0.041645,0.073893,0.025533,3.709691e-02,0.065155,2.521314e-02,3.844909e-02,6.439698e-02,0.024244
5,5,225,0.064410,-0.005404,0.012234,0.083176,0.067360,0.026466,0.070134,0.033220,0.020088,0.055780,0.008508,0.012525,0.052652,0.008472,0.013793,-5.871936e-07,-0.000169,-1.014969e-04,-5.182621e-06,-1.285523e-04,-0.000073
6,6,250,0.061997,-0.017393,0.027061,0.018820,-0.001580,0.013451,0.189037,0.155622,0.031786,0.048662,-0.007640,0.024796,0.048832,-0.016709,-0.007211,-1.719546e-09,-0.000007,-2.775066e-06,-8.389908e-07,-2.850800e-06,-0.000007
7,7,275,0.084432,0.128375,0.097800,0.109516,0.192547,0.108320,0.102921,0.176972,0.108397,0.073269,0.144908,0.110993,0.070322,0.143302,0.112889,7.233643e-02,0.144175,1.100276e-01,7.211387e-02,1.423667e-01,0.106699
8,8,300,0.109596,0.041026,0.083551,0.097602,0.058866,0.086605,0.098751,0.054814,0.084547,0.098849,0.040880,0.086885,0.095496,0.042473,0.086713,1.023098e-01,0.039196,8.664547e-02,9.784490e-02,3.844309e-02,0.087651
9,9,325,0.111924,0.042263,-0.038793,0.133787,0.091224,-0.039374,0.112554,0.067039,-0.034580,0.106888,0.049632,-0.052411,0.104488,0.049346,-0.055748,9.684091e-02,0.041965,-6.858033e-02,1.009017e-01,4.594199e-02,-0.066538


### 5. Fit the model(s) for one window and explore results

In [ ]:

temp_cols = ['PERMNO', 'year', 'prd']
df.reset_index(inplace=True, drop=True)
X = df.copy()
y = X.pop('RET')

train_indx = X.prd<(min_prd+windows_width-1)
val_indx = X['prd'].isin(range(min_prd+windows_width-1, min_prd+windows_width+2))
val_indx_extra = X['prd'].isin(range(min_prd+windows_width+5, min_prd+windows_width+8))
test_indx = X['prd'].isin(range(min_prd+windows_width+2, min_prd+windows_width+5))

X_train = X[train_indx]
X_val = X[val_indx]
X_val_extra = X[val_indx_extra]
X_test = X[test_indx]
y_train = y[train_indx]
y_val = y[val_indx]
y_val_extra = y[val_indx_extra]
y_test = y[test_indx]

#display(X_train.head(3), X_train.tail(3), y_train.head(3), y_train.tail(3))
#display(X_train.prd.describe(), X_val.prd.describe(), X_test.prd.describe())
display(X_train.shape, X_val.shape, X_test.shape)

X_train.drop(columns=temp_cols, inplace=True)
X_val.drop(columns=temp_cols, inplace=True)
X_val_extra.drop(columns=temp_cols, inplace=True)
X_test.drop(columns=temp_cols, inplace=True)

#display(X_train.tail())
col_cat = ['ind']
col_num = [x for x in X_train.columns if x not in col_cat]
for col in col_num:
    X_train[col] = X_train[col].fillna(X_train[col].median())
    X_val[col] = X_val[col].fillna(X_train[col].median())
    X_val_extra[col] = X_val_extra[col].fillna(X_train[col].median())
    X_test[col] = X_test[col].fillna(X_train[col].median())
for col in col_cat:
    X_train[col] = X_train[col].fillna(value=-1000)
    X_val[col] = X_val[col].fillna(value=-1000)
    X_val_extra[col] = X_val_extra[col].fillna(value=-1000)
    X_test[col] = X_test[col].fillna(value=-1000)

#display(X_train.tail())
feature_transformer = ColumnTransformer([('num', StandardScaler(), col_num),
                                        ("cat", OneHotEncoder(sparse = False, handle_unknown="ignore", drop='if_binary'), col_cat)], 
                                        remainder="passthrough")

print('Number of features before transformation: ', X_train.shape)
train_index, val_index, val_index_extra, test_index = X_train.index, X_val.index, X_val_extra.index, X_test.index
X_train = pd.DataFrame(feature_transformer.fit_transform(X_train), columns=feature_transformer.get_feature_names_out())
X_val = pd.DataFrame(feature_transformer.transform(X_val), columns=feature_transformer.get_feature_names_out())
X_val_extra = pd.DataFrame(feature_transformer.transform(X_val_extra), columns=feature_transformer.get_feature_names_out())
X_test = pd.DataFrame(feature_transformer.transform(X_test), columns=feature_transformer.get_feature_names_out())
print('time to do feature proprocessing: ')
print('Number of features after transformation: ', X_train.shape, X_val.shape, X_val_extra.shape, X_test.shape)
X_train.index = train_index
X_val.index = val_index
X_val_extra.index = val_index_extra
X_test.index = test_index
#display(X_train.tail())

X = pd.concat([X_train, X_val])
y = pd.concat([y_train, y_val])
#display(X,y)

X_ = pd.concat([X_train, X_val, X_val_extra])
y_ = pd.concat([y_train, y_val, y_val_extra])
#display(X,y, X_,y_)


results = pd.DataFrame(columns = ['min_prd', 'xgbf_train', 'xgbf_val', 'xgbf_test', 
                                  'xgbgs_train', 'xgbgs_val', 'xgbgs_test', 
                                  'xgbo_train', 'xgbo_val', 'xgbo_test',
                                  'nn4_train', 'nn4_val', 'nn4_test',
                                 'nn6_train', 'nn6_val', 'nn6_test',
                                 'nn4opt_train', 'nn4opt_val', 'nn4opt_test',
                                 'nn6opt_train', 'nn6opt_val', 'nn6opt_test'])



### Modeling part ###

print('mae of a constant model', mean_absolute_error(df.RET, np.ones(df.shape[0])*(df.RET.mean())))
print('R2 of a constant model', r2_score(df.RET, np.ones(df.shape[0])*(df.RET.mean())))

xgb1 = XGBRegressor(tree_method = 'gpu_hist', n_estimators=400, max_depth=4, eta=0.02, colsample_bytree=0.4, subsample=0.6)
xgb1.fit(X_train, y_train)
print('fixed XGB train:', mean_absolute_error(y_train, xgb1.predict(X_train)), r2_score(y_train, xgb1.predict(X_train)))
print('XGB val:', mean_absolute_error(y_val, xgb1.predict(X_val)), r2_score(y_val, xgb1.predict(X_val)))
print('XGB val extra:', mean_absolute_error(y_val_extra, xgb1.predict(X_val_extra)), r2_score(y_val_extra, xgb1.predict(X_val_extra)))
print('XGB test:', mean_absolute_error(y_test, xgb1.predict(X_test)), r2_score(y_test, xgb1.predict(X_test)))

results.loc[results.min_prd==min_prd,'xgbf_train':'xgbf_test'] = \
[r2_score(y_train, xgb1.predict(X_train)), 
r2_score(y_val, xgb1.predict(X_val)),
r2_score(y_test, xgb1.predict(X_test))]

time1 = time.time()

# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if x in X_train.index else 0 for x in X.index]
pds = PredefinedSplit(test_fold = split_index)

xgb = XGBRegressor(tree_method = 'gpu_hist')
param_grid = {'n_estimators':[400, 600, 800], 'max_depth':[2,3,4,5], 'eta':[0.006, 0.012, 0.02], 
              'subsample':[0.6], 'colsample_bytree':[0.6]}
xgbgs = GridSearchCV(estimator = xgb, cv=pds, param_grid=param_grid)

# Fit with all data
xgbgs.fit(X_, y_)

print('gs XGB', xgbgs.best_params_, xgbgs.best_score_, time.time()-time1)
print('XGB train:', mean_absolute_error(y_train, xgbgs.predict(X_train)), r2_score(y_train, xgbgs.predict(X_train)))
print('XGB validation:', mean_absolute_error(y_val, xgbgs.predict(X_val)), r2_score(y_val, xgbgs.predict(X_val)))
print('XGB validation extra:', mean_absolute_error(y_val_extra, xgbgs.predict(X_val_extra)), r2_score(y_val_extra, xgbgs.predict(X_val_extra)))
print('XGB test:', mean_absolute_error(y_test, xgbgs.predict(X_test)), r2_score(y_test, xgbgs.predict(X_test)))

results.loc[results.min_prd==min_prd,'xgbgs_train':'xgbgs_test'] = \
[r2_score(y_train, xgbgs.predict(X_train)), 
r2_score(y_val, xgbgs.predict(X_val)),
r2_score(y_test, xgbgs.predict(X_test))]

time1 = time.time()
def objective_xgb(trial, cv_runs=1, n_splits=2, n_jobs=-1):

    params = {
    "tree_method": 'gpu_hist',
    "verbosity": 0,  # 0 (silent) - 3 (debug)
    "n_estimators": trial.suggest_int("n_estimators", 800, 1500),
    "max_depth": trial.suggest_int("max_depth", 2, 6),
    "learning_rate": trial.suggest_uniform("learning_rate", 0.0005, 0.03),
    "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.05, 0.95),
    "subsample": trial.suggest_uniform("subsample", 0.1, 0.95),
    "alpha": trial.suggest_loguniform("alpha", 0.1, 50.0),
    "lambda": trial.suggest_loguniform("lambda", 0.1, 500.0),
    "gamma": trial.suggest_loguniform("gamma", 1e-10, 100.0),
    "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 50)    }

    model = XGBRegressor(**params, njobs=-1)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose = False)

    score_train = r2_score(y_train, model.predict(X_train))
    score_val = r2_score(y_val, model.predict(X_val))
    score_val_extra = r2_score(y_val_extra, model.predict(X_val_extra)) 
    score_val = (score_val+score_val_extra)/2
    overfit = np.abs(score_train-score_val)

    return score_val-cv_xgb_regularizer*overfit


study = optuna.create_study(direction="maximize")
study.optimize(objective_xgb, n_trials=optuna_xgb_trials)
print('Total time for hypermarameter optimization, XGB: ', time.time()-time1)
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")
optuna_hyperpars = study.best_params
optuna_hyperpars['tree_method']='gpu_hist'
optuna_xgb = XGBRegressor(**optuna_hyperpars)
optuna_xgb.fit(X, y)
print('Optuna XGB train: \n', 
      mean_absolute_error(y_train, optuna_xgb.predict(X_train)), r2_score(y_train, optuna_xgb.predict(X_train)), '\nvalidation \n',
      mean_absolute_error(y_val, optuna_xgb.predict(X_val)), r2_score(y_val, optuna_xgb.predict(X_val)),
      mean_absolute_error(y_val_extra, optuna_xgb.predict(X_val_extra)), r2_score(y_val_extra, optuna_xgb.predict(X_val_extra)), '\ntest \n',
      mean_absolute_error(y_test, optuna_xgb.predict(X_test)), r2_score(y_test, optuna_xgb.predict(X_test)))

results.loc[results.min_prd==min_prd,'xgbo_train':'xgbo_test'] = \
[r2_score(y_train, optuna_xgb.predict(X_train)), 
r2_score(y_val, optuna_xgb.predict(X_val)),
r2_score(y_test, optuna_xgb.predict(X_test))]

###########
### NNs ###
###########

neurons_base = 8
l2_reg_rate = 0.5

model_snn6 = Sequential([
    tf.keras.layers.Dense(units=neurons_base*4, activation="selu", kernel_initializer="lecun_normal", 
                          kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate), input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    tf.keras.layers.Dense(units=neurons_base/2, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    Dense(1)])

print(model_snn6.count_params())

early_stopping50 = EarlyStopping(patience=50, restore_best_weights=True)
optimizer_adam = tf.keras.optimizers.Adam()

model_snn6.compile(loss= "mean_squared_error" , optimizer=optimizer_adam, metrics=["mean_squared_error"])

time1 = time.time()
history = model_snn6.fit(X_train, y_train, validation_data=(X_val, y_val), 
                         batch_size=2048, epochs=1000, verbose=0, callbacks=[early_stopping50])
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

results.loc[results.min_prd==min_prd,'nn6_train':'nn6_test'] = \
[r2_score(y_train, model_snn6.predict(X_train)), 
r2_score(y_val, model_snn6.predict(X_val)),
r2_score(y_test, model_snn6.predict(X_test))]



neurons_base = 8
l2_reg_rate = 0.3

model_snn4 = Sequential([
    tf.keras.layers.Dense(units=neurons_base*4, activation="selu", kernel_initializer="lecun_normal", 
                          kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate), input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(units=neurons_base*2, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    tf.keras.layers.Dense(units=neurons_base, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    tf.keras.layers.Dense(units=neurons_base/2, activation="selu", kernel_initializer="lecun_normal",
                         kernel_regularizer=tf.keras.regularizers.l2(l=l2_reg_rate)),
    Dense(1)])

print(model_snn4.count_params())

time1 = time.time()
model_snn4.compile(loss= "mean_squared_error" , optimizer=optimizer_adam, metrics=["mean_squared_error"])
history = model_snn4.fit(X_train, y_train, validation_data=(X_val, y_val), 
                         batch_size=2048, epochs=1000, verbose=0, callbacks=[early_stopping50])
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

results.loc[results.min_prd==min_prd,'nn4_train':'nn4_test'] = \
[r2_score(y_train, model_snn4.predict(X_train)), 
r2_score(y_val, model_snn4.predict(X_val)),
r2_score(y_test, model_snn4.predict(X_test))]



# try optuna, using this kaggle notebook: https://www.kaggle.com/code/mistag/keras-model-tuning-with-optuna

time1 = time.time()
study = optuna.create_study(direction="minimize", 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective_nn4, n_trials=optuna_nn_trials)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
temp = study.best_params
display('Optuna NN4', study.best_params, time.time()-time1)

optimal_hyperpars = list(temp.values())
display(optimal_hyperpars)
print('Time for hyperparameter optimization: ',time.time()-time1, optimal_hyperpars)

optuna_nn = create_snnn4_model_hyperpars(neurons_base=optimal_hyperpars[0], 
                                         l2_reg_rate=optimal_hyperpars[1],
                                        l1_reg_rate=optimal_hyperpars[2])
history = optuna_nn.fit(X_train, y_train, 
                        validation_data=(X_val, y_val),
                        batch_size=2048, 
                        epochs=1000,
                        verbose=0, 
                        callbacks=[early_stopping50])

results.loc[results.min_prd==min_prd,'nn4opt_train':'nn4opt_test'] = \
[r2_score(y_train, optuna_nn.predict(X_train)), 
r2_score(y_val, optuna_nn.predict(X_val)),
r2_score(y_test, optuna_nn.predict(X_test))]


time1 = time.time()
study = optuna.create_study(direction="minimize", 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective_nn6, n_trials=optuna_nn_trials)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
temp = study.best_params
display('Optuna NN6', study.best_params, time.time()-time1)

optimal_hyperpars = list(temp.values())
display(optimal_hyperpars)
print('Time for hyperparameter optimization: ', time.time()-time1, optimal_hyperpars)

optuna_nn = create_snnn6_model_hyperpars(neurons_base=optimal_hyperpars[0], 
                                         l2_reg_rate=optimal_hyperpars[1],
                                        l1_reg_rate=optimal_hyperpars[2])
history = optuna_nn.fit(X_train, y_train, 
                        validation_data=(X_val, y_val),
                        batch_size=2048, 
                        epochs=1000,
                        verbose=0, 
                        callbacks=[early_stopping50])

results.loc[results.min_prd==min_prd,'nn6opt_train':'nn6opt_test'] = \
[r2_score(y_train, optuna_nn.predict(X_train)), 
r2_score(y_val, optuna_nn.predict(X_val)),
r2_score(y_test, optuna_nn.predict(X_test))]


print('total time for the script: ', time.time()-time0)
display(results)

In [ ]:
### Feature Importance

In [ ]:
### Error Analysis

In [ ]:
# general point:
# compared to NN, xgb is harder to regularize
# in NN, you can simply shrink coefficient towards constant prediction.
# in xgb, you can not do that. the only way to regularize is via hyperparameters.
# in other words, by tweaking hyperpars, in NN you can approach R^2=0.0 prediction from a constant model arbitrarily close
# in xgb, you can not do that.
# by setting eta as low as 0.1% you can bring r2 down to 0.1%, but lowering eta further actyally increases abs(r2).


In [ ]:
print('total time for the script: ', time.time()-time0)
display(results)

In [ ]:
 
# def objective_nn(trial):
    
#     tf.keras.backend.clear_session()
    
#     with strategy.scope():
#         # Generate our trial model.
#         model = create_snnn_model(trial)

#         callbacks = [
#         tf.keras.callbacks.EarlyStopping(patience=40),
#         TFKerasPruningCallback(trial, "val_loss"),
#     ]

#         # Fit the model on the training data.
#         # The TFKerasPruningCallback checks for pruning condition every epoch.
        
#         history = model.fit(X_train, y_train, 
#                                 validation_data=(X_val, y_val),
#                                 batch_size=2048, 
#                                 epochs=500, 
#                                 verbose=1, 
#                                 callbacks=callbacks)

#         # Evaluate the model accuracy on the validation set.
#         score = model.evaluate(X_val, y_val, verbose=0)
#         return score[1]

# trials = 50

# study = optuna.create_study(direction="minimize", 
#                             sampler=optuna.samplers.TPESampler(), 
#                             pruner=optuna.pruners.HyperbandPruner())
# study.optimize(objective_nn, n_trials=trials)
# pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
# complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
# temp = study.best_params
# display(study.best_params, time.time()-time1)

# optimal_hyperpars = list(temp.values())
# display(optimal_hyperpars)
# print(time.time()-time1, optimal_hyperpars)

# optuna_nn = create_snnn_model_hyperpars(neurons_base=optimal_hyperpars[0], l2_reg_rate=optimal_hyperpars[1])
# history = optuna_nn.fit(X_train, y_train, 
#                         validation_data=(X_val, y_val),
#                         batch_size=2048, 
#                         epochs=1000,
#                         verbose=1, 
#                         callbacks=[early_stopping50])

# results.loc[results.min_prd==min_prd,'nn4opt_train':'nn4opt_test'] = \
# [r2_score(y_train, optuna_nn.predict(X_train)), 
# r2_score(y_val, optuna_nn.predict(X_val)),
# r2_score(y_test, optuna_nn.predict(X_test))]

In [ ]:
# # try optuna for NN:

# def objective(trial):

#     n_layers = trial.suggest_int('n_layers', 1, 3)
#     model = tf.keras.Sequential()
#     for i in range(n_layers):
#         num_hidden = trial.suggest_int(f'n_units_l{i}', 4, 128, log=True)
#         model.add(tf.keras.layers.Dense(num_hidden, activation='relu'))
#     model.add(tf.keras.layers.Dense(1))
#     display(model.summary())
#     return accuracy

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10)


In [ ]:
# on nns:
# - try classic regularizers (l1, l2 etc)
# - try different architecture (not snnn)
# classic architecture:
# He initialization, elu activation, batch norm, l2 reg, adam.

# - try exotic architecture, e.g., wide'n'deep
# 


In [ ]:
# usually self-norm seems better: it overfits less and runs faster

